In [3]:
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np

In [4]:
# 1. Path to SQLite
db_path = Path("data") / "processed" / "data_processed.sqlite"

# 2. Load table
with sqlite3.connect(db_path) as conn:
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
    if tables.empty:
        raise RuntimeError(f"No tables found in {db_path}")
    table_name = tables["name"].iloc[0]
    df = pd.read_sql(f'SELECT * FROM "{table_name}"', conn, parse_dates=["Date"])

df = df.sort_values("Date").set_index("Date")

# 3. Ensure required columns exist
for col in ["Close", "SMA50", "SMA200"]:
    if col not in df.columns:
        raise KeyError(f"Required column '{col}' not found; columns = {df.columns.tolist()}")

# 4. SMA50 > SMA200 strategy: signal = 1 if SMA50 > SMA200, else -1
df["signalSMA"] = np.where(df["SMA50"] > df["SMA200"], 1, -1)

# 5. Returns based on signal
df["market_ret"]   = df["Close"].pct_change().fillna(0)
df["strategy_ret"] = df["signalSMA"].shift(1).fillna(0) * df["market_ret"]
df["strategy_eq"]  = (1 + df["strategy_ret"]).cumprod()

print(df[["Close", "SMA50", "SMA200", "signalSMA"]].tail())

# 6. Save signals to SQLite
out_db_path = Path("data") / "signals" / "SMAsignal.sqlite"
out_db_path.parent.mkdir(parents=True, exist_ok=True)

with sqlite3.connect(out_db_path) as conn:
    df.reset_index().to_sql("signals_sma", conn, if_exists="replace", index=False)

print("Saved SMA signals to:", out_db_path.resolve())

                 Close       SMA50      SMA200  signalSMA
Date                                                     
2025-12-01  283.100006  263.215672  226.971342          1
2025-12-02  286.190002  263.822832  227.183460          1
2025-12-03  284.149994  264.422160  227.386025          1
2025-12-04  280.700012  264.994847  227.569348          1
2025-12-05  278.779999  265.438023  227.738286          1
Saved SMA signals to: C:\Users\soltv\Documents\GitHub\newalgotrade\data\signals\SMAsignal.sqlite
